In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from time import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

In [80]:
import urllib.request  # the lib that handles the url stuff

target_url = 'https://raw.githubusercontent.com/sepehrilami/DataAnalysisInternship-AIMed/master/RNN/ferdosi.txt'
data = urllib.request.urlopen(target_url)
lines = []
for line in data:
    lines.append(line.decode('utf-8'))

print(len(lines))

49609


In [81]:
SOS_token = 0
EOS_token = 1

class Language:

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.num_of_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_of_words
            self.word2count[word] = 1
            self.index2word[self.num_of_words] = word
            self.num_of_words += 1
        else:
            self.word2count[word] += 1
    
        # Find letter index from all_letters, e.g. "a" = 0
        def letterToIndex(letter):
            return all_letters.find(letter)

        # Just for demonstration, turn a letter into a <1 x n_letters> Tensor
        def letterToTensor(letter):
            tensor = torch.zeros(1, n_letters)
            tensor[0][letterToIndex(letter)] = 1
            return tensor

        # Turn a line into a <line_length x 1 x n_letters>,
        # or an array of one-hot letter vectors
        def lineToTensor(line):
            tensor = torch.zeros(len(line), 1, n_letters)
            for li, letter in enumerate(line):
                tensor[li][0][letterToIndex(letter)] = 1
            return tensor    

In [82]:
language = Language('Persian')

In [135]:
for line in lines:
    language.addSentence(line)

pair = [[word for word in line] for line in lines]
pairs = [pair, pair]

In [136]:
print(language.num_of_words)

20387


In [137]:
class EncoderRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, device):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.device = device

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [138]:
# FloatTensor containing pretrained weights
embedding = torch.nn.Embedding(4, 2)
# Get embeddings for index 1
input = torch.LongTensor([[1], [3]])
input

tensor([[1],
        [3]])

In [139]:
embedding(input).shape

torch.Size([2, 1, 2])

In [140]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [141]:
MAX_LENGTH = 10

In [142]:

class AttnDecoderRNN(torch.nn.Module):
    def __init__(self, hidden_size, output_size, device, dropout_p=0.1, max_length= MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.device = device

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [143]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(language, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [144]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [145]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [155]:
from torch import optim

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)
    training_verses = [tensorFromSentence(language, np.random.choice(lines)) for i in range(n_iters)]
    criterion = torch.nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        input_tensor = training_verses[iter - 1]
        target_tensor = input_tensor

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [156]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [160]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [162]:
hidden_size = 256
encoder1 = EncoderRNN(language.num_of_words, hidden_size, device=device)
attn_decoder1 = AttnDecoderRNN(hidden_size, language.num_of_words, device=device, dropout_p=0.1)

trainIters(encoder1, attn_decoder1, int(len(lines) * 0.8), print_every=1)

RuntimeError: ignored